In [1]:
!pip install ollama duckduckgo_search
!pip install langchain
!pip install flask ollama duckduckgo_search

In [6]:
from flask import Flask, request, jsonify, render_template_string
from ollama import chat
import ollama

app = Flask(__name__)

@app.route('/')
def index():
    # 사용 가능한 모델 목록 가져오기
    try:
        models_response = ollama.list()
        available_models = [model['name'] for model in models_response['models']]
    except:
        available_models = ['llama3.2', 'llama2']  # 기본값
    
    html = """
<!DOCTYPE html>
<html lang="en" class="dark">
<head>
    <meta charset="UTF-8">
    <title>SHPGPT</title>
    <script src="https://cdn.tailwindcss.com"></script>
</head>
<body class="bg-black text-white">
<div class="flex h-screen">
    <!-- Sidebar -->
    <div class="w-64 border-r border-gray-700 flex flex-col p-4 space-y-2">
        <h1 class="text-2xl font-bold mb-2">SHPGPT</h1>
        <button class="bg-gray-800 rounded px-2 py-1 text-left text-sm hover:bg-gray-700" onclick="newChat()">new chat</button>
        <h2 class="mt-4 mb-2 text-sm text-gray-400">chat history</h2>
        <div class="space-y-1 flex-1 overflow-y-auto">
            <!-- Sample chats -->
            <div class="bg-gray-800 rounded px-2 py-1 text-sm hover:bg-gray-700 cursor-pointer">Chat 1</div>
            <div class="bg-gray-800 rounded px-2 py-1 text-sm hover:bg-gray-700 cursor-pointer">Chat 2</div>
        </div>
    </div>

    <!-- Main Chat Area -->
    <div class="flex-1 flex flex-col">
        <div class="flex-1 overflow-y-auto p-6 space-y-4" id="chat-window">
            <div class="text-center text-gray-400 mt-20">
                <h2 class="text-2xl mb-2">Welcome to SHPGPT</h2>
                <p>Start a conversation by typing your message below.</p>
            </div>
        </div>

        <!-- Prompt Input -->
        <div class="border-t border-gray-700 p-4 flex items-end space-x-2">
            <textarea 
                id="prompt" 
                rows="2" 
                placeholder="type prompt here" 
                class="flex-1 p-2 rounded bg-gray-900 border border-gray-700 text-white resize-none"
                onkeydown="handleKeyDown(event)"
            ></textarea>
            <select id="model-select" class="bg-gray-800 text-white px-2 py-1 rounded border border-gray-700">
                {% for model in models %}
                    <option value="{{ model }}">{{ model }}</option>
                {% endfor %}
            </select>
            <button 
                id="send-btn" 
                class="bg-blue-600 px-4 py-2 rounded text-white hover:bg-blue-700 disabled:bg-gray-600" 
                onclick="sendMessage()"
            >
                ↑
            </button>
        </div>
    </div>
</div>

<script>
let isLoading = false;

function handleKeyDown(event) {
    if (event.key === 'Enter' && !event.shiftKey) {
        event.preventDefault();
        sendMessage();
    }
}

function newChat() {
    document.getElementById('chat-window').innerHTML = `
        <div class="text-center text-gray-400 mt-20">
            <h2 class="text-2xl mb-2">Welcome to SHPGPT</h2>
            <p>Start a conversation by typing your message below.</p>
        </div>
    `;
}

async function sendMessage() {
    if (isLoading) return;
    
    const prompt = document.getElementById('prompt').value.trim();
    const model = document.getElementById('model-select').value;
    const sendBtn = document.getElementById('send-btn');
    const chatWindow = document.getElementById('chat-window');

    if (!prompt) return;

    // UI 업데이트
    isLoading = true;
    sendBtn.disabled = true;
    sendBtn.textContent = '...';

    // 사용자 메시지 추가
    const userMessage = document.createElement('div');
    userMessage.className = 'flex justify-end mb-4';
    userMessage.innerHTML = `
        <div class='bg-blue-700 px-3 py-2 rounded max-w-lg break-words'>
            ${escapeHtml(prompt)}
        </div>
    `;
    chatWindow.appendChild(userMessage);

    // AI 응답 준비 (로딩 표시)
    const aiMessageContainer = document.createElement('div');
    aiMessageContainer.className = 'flex justify-start mb-4';
    aiMessageContainer.innerHTML = `
        <div class='bg-gray-700 px-3 py-2 rounded max-w-lg'>
            <div class="animate-pulse">Thinking...</div>
        </div>
    `;
    chatWindow.appendChild(aiMessageContainer);

    // 입력창 초기화 및 스크롤
    document.getElementById('prompt').value = '';
    chatWindow.scrollTop = chatWindow.scrollHeight;

    try {
        const response = await fetch('/ask', {
            method: 'POST',
            headers: { 'Content-Type': 'application/json' },
            body: JSON.stringify({ prompt, model })
        });

        if (!response.ok) {
            throw new Error(`HTTP error! status: ${response.status}`);
        }

        const data = await response.json();
        
        // AI 응답 업데이트
        aiMessageContainer.innerHTML = `
            <div class='bg-gray-700 px-3 py-2 rounded max-w-lg break-words'>
                ${escapeHtml(data.response || data.message || 'No response received')}
            </div>
        `;

        // Hidden thinking이 있다면 표시 (옵션)
        if (data.hidden) {
            const hiddenDiv = document.createElement('div');
            hiddenDiv.className = 'text-xs text-gray-500 mt-1 italic';
            hiddenDiv.textContent = `Thinking: ${data.hidden}`;
            aiMessageContainer.appendChild(hiddenDiv);
        }

    } catch (error) {
        console.error('Error:', error);
        aiMessageContainer.innerHTML = `
            <div class='bg-red-700 px-3 py-2 rounded max-w-lg'>
                Error: ${error.message}
            </div>
        `;
    } finally {
        isLoading = false;
        sendBtn.disabled = false;
        sendBtn.textContent = '↑';
        chatWindow.scrollTop = chatWindow.scrollHeight;
    }
}

function escapeHtml(text) {
    const div = document.createElement('div');
    div.textContent = text;
    return div.innerHTML;
}

// 페이지 로드 시 초기화
document.addEventListener('DOMContentLoaded', function() {
    document.getElementById('prompt').focus();
});
</script>
</body>
</html>
    """
    return render_template_string(html, models=available_models)

@app.route('/ask', methods=['POST'])
def ask():
    data = request.get_json()
    
    if not data:
        return jsonify({"error": "No data provided"}), 400
        
    user_message = data.get('prompt') or data.get('message', '')
    model = data.get('model', 'llama3.2')
    
    if not user_message.strip():
        return jsonify({"error": "Empty message"}), 400
    
    try:
        response = chat(model=model, messages=[{'role': 'user', 'content': user_message}])
        answer = response.message.content
        
        # 응답 내에 /think 태그가 있으면 visible 메시지와 hidden 추론 내용으로 분리
        hidden = ""
        visible = answer
        
        if "/think" in answer:
            parts = answer.split("/think")
            if len(parts) >= 3:
                visible = parts[0].strip()
                hidden = parts[1].strip()
            
        return jsonify({
            "response": visible,
            "hidden": hidden,
            "model": model
        })
        
    except Exception as e:
        error_msg = f"Model error: {str(e)}"
        print(f"Error occurred: {error_msg}")  # 서버 로그에 출력
        return jsonify({"error": error_msg}), 500

if __name__ == "__main__":
    print("Starting SHPGPT server...")
    print("Access the app at: http://127.0.0.1:5000")
    app.run(host="127.0.0.1", port=5000, debug=True, use_reloader=False)

Starting SHPGPT server...
Access the app at: http://127.0.0.1:5000
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


In [5]:
from flask import Flask, request, jsonify, render_template_string
from ollama import chat
import ollama

app = Flask(__name__)

@app.route('/')
def index():
    # 사용 가능한 모델 목록
    available_models = [
        "qwen3:latest", "qwen2.5vl:latest", "phi4-mini-reasoning:latest",
        "huihui_ai/qwen3-abliterated:4b-q4_K_M", "qwen3:0.6b",
        "gemma3:latest", "qwen3:4b", "gemma3:1b", "phi4-mini:latest"
    ]
    
    html = """
<!DOCTYPE html>
<html lang="en" class="dark">
<head>
    <meta charset="UTF-8">
    <title>SHPGPT</title>
    <script src="https://cdn.tailwindcss.com"></script>
</head>
<body class="bg-black text-white">
<div class="flex h-screen">
    <!-- Sidebar -->
    <div class="w-64 border-r border-gray-700 flex flex-col p-4 space-y-2">
        <h1 class="text-2xl font-bold mb-2">SHPGPT</h1>
        <button class="bg-gray-800 rounded-xl px-3 py-2 text-left text-sm hover:bg-gray-700" onclick="newChat()">new chat</button>
        <h2 class="mt-4 mb-2 text-sm text-gray-400">chat history</h2>
        <div class="space-y-1 flex-1 overflow-y-auto" id="chat-history">
            <!-- Chat history will be populated here -->
        </div>
    </div>

    <!-- Main Chat Area -->
    <div class="flex-1 flex flex-col">
        <div class="flex-1 overflow-y-auto p-6 space-y-4" id="chat-window">
            <div class="text-center text-gray-400 mt-20">
                <h2 class="text-2xl mb-2">Welcome to SHPGPT</h2>
                <p>Start a conversation by typing your message below.</p>
            </div>
        </div>

        <!-- Prompt Input -->
        <div class="border-t border-gray-700 p-4 flex items-end space-x-3">
            <textarea 
                id="prompt" 
                rows="2" 
                placeholder="type prompt here" 
                class="flex-1 p-3 rounded-2xl bg-gray-900 border border-gray-700 text-white resize-none"
                onkeydown="handleKeyDown(event)"
            ></textarea>
            <select id="model-select" class="bg-gray-800 text-white px-3 py-2 rounded-xl border border-gray-700">
                {% for model in models %}
                    <option value="{{ model }}" {% if model == "qwen3:0.6b" %}selected{% endif %}>{{ model }}</option>
                {% endfor %}
            </select>
            <button 
                id="send-btn" 
                class="bg-violet-500 px-4 py-2 rounded-xl text-white hover:bg-violet-400 disabled:bg-gray-600" 
                onclick="sendMessage()"
            >
                ↑
            </button>
        </div>
    </div>
</div>

<script>
let isLoading = false;
let currentChatId = null;
let chatHistory = [];

// 채팅 제목 생성 함수
async function generateChatTitle(firstMessage) {
    try {
        const response = await fetch('/generate_title', {
            method: 'POST',
            headers: { 'Content-Type': 'application/json' },
            body: JSON.stringify({ message: firstMessage })
        });
        
        if (response.ok) {
            const data = await response.json();
            return data.title || 'New Chat';
        }
    } catch (error) {
        console.error('Title generation failed:', error);
    }
    return 'New Chat';
}

// 채팅 히스토리에 추가
function addToChatHistory(chatId, title) {
    const historyContainer = document.getElementById('chat-history');
    const chatItem = document.createElement('div');
    chatItem.className = 'bg-gray-800 rounded-xl px-3 py-2 text-sm hover:bg-gray-700 cursor-pointer';
    chatItem.textContent = title;
    chatItem.onclick = () => loadChat(chatId);
    historyContainer.appendChild(chatItem);
}

function handleKeyDown(event) {
    if (event.key === 'Enter' && !event.shiftKey) {
        event.preventDefault();
        sendMessage();
    }
}

function newChat() {
    currentChatId = null;
    document.getElementById('chat-window').innerHTML = `
        <div class="text-center text-gray-400 mt-20">
            <h2 class="text-2xl mb-2">Welcome to SHPGPT</h2>
            <p>Start a conversation by typing your message below.</p>
        </div>
    `;
}

function loadChat(chatId) {
    // TODO: 실제 채팅 로드 구현
    console.log('Loading chat:', chatId);
}

async function sendMessage() {
    if (isLoading) return;
    
    const prompt = document.getElementById('prompt').value.trim();
    const model = document.getElementById('model-select').value;
    const sendBtn = document.getElementById('send-btn');
    const chatWindow = document.getElementById('chat-window');

    if (!prompt) return;

    // UI 업데이트
    isLoading = true;
    sendBtn.disabled = true;
    sendBtn.textContent = '...';

    // 사용자 메시지 추가
    const userMessage = document.createElement('div');
    userMessage.className = 'flex justify-end mb-4';
    userMessage.innerHTML = `
        <div class='bg-violet-500 px-4 py-3 rounded-2xl max-w-lg break-words'>
            ${escapeHtml(prompt)}
        </div>
    `;
    chatWindow.appendChild(userMessage);

    // AI 응답 준비 (로딩 표시)
    const aiMessageContainer = document.createElement('div');
    aiMessageContainer.className = 'flex justify-start mb-4';
    aiMessageContainer.innerHTML = `
        <div class='bg-gray-700 px-4 py-3 rounded-2xl max-w-lg'>
            <div class="animate-pulse">Thinking...</div>
        </div>
    `;
    chatWindow.appendChild(aiMessageContainer);

    // 입력창 초기화 및 스크롤
    document.getElementById('prompt').value = '';
    chatWindow.scrollTop = chatWindow.scrollHeight;

    // 첫 번째 메시지인 경우 채팅 제목 생성
    const isFirstMessage = document.querySelectorAll('#chat-window .flex').length === 1;
    
    try {
        const response = await fetch('/ask', {
            method: 'POST',
            headers: { 'Content-Type': 'application/json' },
            body: JSON.stringify({ prompt, model })
        });

        if (!response.ok) {
            throw new Error(`HTTP error! status: ${response.status}`);
        }

        const data = await response.json();
        
        // AI 응답 업데이트
        aiMessageContainer.innerHTML = `
            <div class='bg-gray-700 px-4 py-3 rounded-2xl max-w-lg break-words'>
                ${escapeHtml(data.response || data.message || 'No response received')}
            </div>
        `;

        // Hidden thinking이 있다면 접을 수 있는 블록으로 표시
        if (data.hidden && data.hidden.trim()) {
            const thinkingContainer = document.createElement('div');
            thinkingContainer.className = 'mt-2';
            
            const thinkingToggle = document.createElement('button');
            thinkingToggle.className = 'flex items-center space-x-2 text-xs text-violet-400 hover:text-violet-300 focus:outline-none';
            thinkingToggle.innerHTML = `
                <span class="toggle-icon">▶</span>
                <span>💭 Show thinking</span>
                <span class="thinking-time text-gray-500">(${data.thinking_time || 'N/A'})</span>
            `;
            
            const thinkingContent = document.createElement('div');
            thinkingContent.className = 'bg-gray-800 border-l-4 border-violet-500 px-4 py-3 mt-2 rounded-xl text-sm text-gray-300 hidden';
            thinkingContent.innerHTML = `<div>${escapeHtml(data.hidden)}</div>`;
            
            thinkingToggle.addEventListener('click', function() {
                const icon = this.querySelector('.toggle-icon');
                const label = this.querySelector('span:nth-child(2)');
                
                if (thinkingContent.classList.contains('hidden')) {
                    thinkingContent.classList.remove('hidden');
                    icon.textContent = '▼';
                    label.textContent = '💭 Hide thinking';
                } else {
                    thinkingContent.classList.add('hidden');
                    icon.textContent = '▶';
                    label.textContent = '💭 Show thinking';
                }
            });
            
            thinkingContainer.appendChild(thinkingToggle);
            thinkingContainer.appendChild(thinkingContent);
            aiMessageContainer.appendChild(thinkingContainer);
        }

        // 첫 번째 메시지인 경우 제목 생성
        if (isFirstMessage) {
            const title = await generateChatTitle(prompt);
            currentChatId = Date.now().toString();
            addToChatHistory(currentChatId, title);
        }

    } catch (error) {
        console.error('Error:', error);
        aiMessageContainer.innerHTML = `
            <div class='bg-red-700 px-4 py-3 rounded-2xl max-w-lg'>
                Error: ${error.message}
            </div>
        `;
    } finally {
        isLoading = false;
        sendBtn.disabled = false;
        sendBtn.textContent = '↑';
        chatWindow.scrollTop = chatWindow.scrollHeight;
    }
}

function escapeHtml(text) {
    const div = document.createElement('div');
    div.textContent = text;
    return div.innerHTML;
}

// 페이지 로드 시 초기화
document.addEventListener('DOMContentLoaded', function() {
    document.getElementById('prompt').focus();
});
</script>
</body>
</html>
    """
    return render_template_string(html, models=available_models)

@app.route('/generate_title', methods=['POST'])
def generate_title():
    data = request.get_json()
    user_message = data.get('message', '')
    
    if not user_message.strip():
        return jsonify({"title": "New Chat"})
    
    try:
        # qwen3:0.6b 모델을 사용해 제목 생성
        title_prompt = f"Generate a short title (max 4-5 words) for a conversation that starts with: '{user_message[:100]}...'. Only respond with the title, nothing else."
        
        response = chat(model='qwen3:0.6b', messages=[{'role': 'user', 'content': title_prompt}])
        title = response.message.content.strip()
        
        # 제목이 너무 길면 자르기
        if len(title) > 30:
            title = title[:27] + "..."
            
        return jsonify({"title": title})
        
    except Exception as e:
        print(f"Title generation error: {str(e)}")
        # 기본 제목 생성 (첫 몇 단어 사용)
        words = user_message.split()[:3]
        fallback_title = " ".join(words)
        if len(fallback_title) > 20:
            fallback_title = fallback_title[:17] + "..."
        return jsonify({"title": fallback_title or "New Chat"})

@app.route('/ask', methods=['POST'])
def ask():
    import time
    start_time = time.time()
    
    data = request.get_json()
    
    if not data:
        return jsonify({"error": "No data provided"}), 400
        
    user_message = data.get('prompt') or data.get('message', '')
    model = data.get('model', 'qwen3:0.6b')  # 기본 모델을 qwen3:0.6b로 변경
    
    if not user_message.strip():
        return jsonify({"error": "Empty message"}), 400
    
    try:
        response = chat(model=model, messages=[{'role': 'user', 'content': user_message}])
        answer = response.message.content
        
        # 응답 내에 <think> 태그가 있으면 visible 메시지와 hidden 추론 내용으로 분리
        hidden = ""
        visible = answer
        thinking_time = None
        
        if "<think>" in answer and "</think>" in answer:
            thinking_start = time.time()
            
            # <think>...</think> 부분을 찾아서 분리
            start_tag = answer.find("<think>")
            end_tag = answer.find("</think>")
            
            if start_tag != -1 and end_tag != -1 and end_tag > start_tag:
                # think 태그 앞의 내용
                before_think = answer[:start_tag].strip()
                # think 태그 안의 내용
                think_content = answer[start_tag + 7:end_tag].strip()  # 7 = len("<think>")
                # think 태그 뒤의 내용
                after_think = answer[end_tag + 8:].strip()  # 8 = len("</think>")
                
                # visible은 think 태그를 제외한 내용
                visible = (before_think + " " + after_think).strip()
                hidden = think_content
                
                # thinking 시간 계산
                thinking_end = time.time()
                thinking_duration = thinking_end - start_time
                thinking_time = f"{thinking_duration:.2f}s"
            
        return jsonify({
            "response": visible,
            "hidden": hidden,
            "thinking_time": thinking_time,
            "model": model
        })
        
    except Exception as e:
        error_msg = f"Model error: {str(e)}"
        print(f"Error occurred: {error_msg}")  # 서버 로그에 출력
        return jsonify({"error": error_msg}), 500

if __name__ == "__main__":
    print("Starting SHPGPT server...")
    print("Access the app at: http://127.0.0.1:5000")
    app.run(host="127.0.0.1", port=5000, debug=True, use_reloader=False)

Starting SHPGPT server...
Access the app at: http://127.0.0.1:5000
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
